# Importaciones

In [2]:
import math 
import numpy as np 

# Paso Nº1:

Create an evaluation matrix consisting of m alternatives and n criteria, with the intersection of each alternative and criteria given as $x_{ij}$, we therefore have a matrix $(x_{ij})_{(m×n)}$.


# Paso Nº2:

The matrix $\left(x_{i j}\right)_{m \times n}$ is then normalised to form the matrix
$R=\left(r_{i j}\right)_{m \times n}$, using the normalisation method

$$
r_{i j}=\frac{x_{i j}}{\sqrt{\sum_{k=1}^{m} x_{k j}^{2}}}, \quad i=1,2, \ldots, m, \quad j=1,2, \ldots, n
$$

In [3]:
def paso2(matrix):
    
    rows, columns = matrix.shape
    
    normal_matrix = np.copy(matrix)
    
    x_kj = np.zeros(columns)
    
    for i in range(rows):
        for j in range(columns):
            x_kj[j] += matrix[i, j]**2
            
    for i in range(rows):
        for j in range(columns):
            normal_matrix[i,j] = matrix[i, j]/(x_kj[j]**0.5)


        
    return normal_matrix

# Paso Nº3

Calculate the weighted normalised decision matrix $t_{i j}=r_{i j} \cdot w_{j}, \quad i=1,2, \ldots, m, \quad j=1,2, \ldots, n$ 


In [4]:
def paso3(matrix, weight):
    rows, columns = matrix.shape
    
    weighted_normalized = np.copy(matrix)
    
    for i in range(rows):
        for j in range(columns):
            weighted_normalized[i, j] *= weight[j]
    
    return weighted_normalized



# Paso Nº4

Determine the worst alternative $\left(A_{w}\right)$ and the best alternative $\left(A_{b}\right)$ :

$$
\begin{aligned}
&A_{w}=\left\{\left\langle\max \left(t_{i j} \mid i=1,2, \ldots, m\right) \mid j \in J_{-}\right\rangle,\left\langle\min \left(t_{i j} \mid i=1,2, \ldots, m\right) \mid j \in J_{+}\right\rangle\right\} \equiv\left\{t_{w j} \mid j=1,2, \ldots, n\right\} \\
&A_{b}=\left\{\left\langle\min \left(t_{i j} \mid i=1,2, \ldots, m\right) \mid j \in J_{-}\right\rangle,\left\langle\max \left(t_{i j} \mid i=1,2, \ldots, m\right) \mid j \in J_{+}\right\rangle\right\} \equiv\left\{t_{b j} \mid j=1,2, \ldots, n\right\}
\end{aligned}
$$

where,
$J_{+}=\{j=1,2, \ldots, n \mid j\}$ associated with the criteria having a positive impact, and

$J_{-}=\{j=1,2, \ldots, n \mid j\}$ associated with the criteria having a negative impact.

In [5]:
def paso4(matrix, criteria):
    
    rows, columns = matrix.shape

    worst_alternatives = np.zeros(columns)
    best_alternatives = np.zeros(columns)
    
    for i in range(columns):
        if(criteria[i]):
            worst_alternatives[i] = min(matrix[:, i])
            best_alternatives[i] = max(matrix[:, i])
        else:
            worst_alternatives[i] = max(matrix[:, i])
            best_alternatives[i] = min(matrix[:, i])
    
    return worst_alternatives, best_alternatives

# Paso Nº5

Calculate the $\mathrm{L}^{2}$-distance between the target alternative $i$ and the worst condition $A_{w}$

$$
d_{i w}=\sqrt{\sum_{j=1}^{n}\left(t_{i j}-t_{w j}\right)^{2}}, \quad i=1,2, \ldots, m
$$

and the distance between the alternative $i$ and the best condition $A_{b}$

$$
d_{i b}=\sqrt{\sum_{j=1}^{n}\left(t_{i j}-t_{b j}\right)^{2}}, \quad i=1,2, \ldots, m
$$

where $d_{i w}$ and $d_{i b}$ are $\mathrm{L}^{2}$-norm distances from the target alternative $i$ to the worst and best conditions, respectively.

In [6]:
def paso5(matrix, worst_alternative, best_alternatives):
    rows, columns = matrix.shape
    
    worst_distance = np.zeros(rows)
    best_distance = np.zeros(rows)

    worst_distance_mat = np.copy(matrix)
    best_distance_mat = np.copy(matrix)

    for i in range(rows):
        for j in range(columns):
            worst_distance_mat[i][j] = (matrix[i][j]-worst_alternatives[j])**2
            best_distance_mat[i][j] = (matrix[i][j]-best_alternatives[j])**2
                
            worst_distance[i] += worst_distance_mat[i][j]
            best_distance[i] += best_distance_mat[i][j]

        for i in range(rows):
            worst_distance[i] = worst_distance[i]**0.5
            best_distance[i] = best_distance[i]**0.5
            
    return worst_distance, best_distance

# Paso Nº6

Calculate the similarity to the worst condition:

$$
s_{i w}=d_{i w} /\left(d_{i w}+d_{i b}\right), \quad 0 \leq s_{i w} \leq 1, \quad i=1,2, \ldots, m .
$$

$s_{i w}=1$ if and only if the alternative solution has the best condition; and

$s_{i w}=0$ if and only if the alternative solution has the worst condition.

In [7]:
def paso6(matrix, worst_distance, best_distance):
    
    rows, columns = matrix.shape   
    
    worst_similarity = np.zeros(rows)
    best_similarity = np.zeros(rows)

    for i in range(rows):
        worst_similarity[i] = worst_distance[i] / \
        (worst_distance[i] + best_distance[i])

        best_similarity[i] = best_distance[i] / \
        (worst_distance[i] + best_distance[i])
        
    return worst_similarity, best_similarity

# Paso Nº7 

Rank the alternatives according to $s_{i w}(i=1,2, \ldots, m)$.

In [8]:
def ranking(data):
    return [i+1 for i in data.argsort()]

def rank_to_worst_similarity(worst_similarity):
    return ranking(worst_similarity)

def rank_to_best_similarity(best_similarity):
    return ranking(best_similarity)

In [9]:
matrix = np.array([
    [70.0 ,40.0 ,70.0 ,25.0, 50.0, 80.0, 100.0, 70.0, 70.0, 50.0],
    [60.0 ,55.0 ,50.0 ,45.0, 50.0, 70.0, 100.0, 80.0, 80.0, 60.0],
    [50.0 ,70.0 ,50.0 ,60.0, 60.0, 70.0, 100.0, 70.0, 30.0, 30.0],
    [80.0 ,90.0 ,20.0 ,30.0, 70.0, 100.0, 100.0, 80.0, 50.0, 40.0]])

weights = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

#Maximizar - True 
#Minimizar - False

criteria = np.array([True, True, True, True, True, True ,True, True, True, False])

normal = paso2(matrix)

weighted = paso3(normal, weights)

worst_alternatives, best_alternatives = paso4(weighted, criteria)

worst_distance, best_distance = paso5(weighted, worst_alternatives, best_alternatives)

worst_similarity, best_similarity = paso6(weighted, worst_distance, best_distance)

rank1 = rank_to_worst_similarity(worst_similarity)

rank2 = rank_to_best_similarity(best_similarity)


In [11]:
print("Results:")

print("Best distance ", best_distance)
print("Worst distance ", worst_distance)

print("--------------------------------------------------------------------")

print("Worst similarity ", worst_similarity)
print("Rank to worst similarity ", rank1)

print("--------------------------------------------------------------------")

print("Best similarity ", best_similarity)
print("Rank to worst similarity ", rank2)

Results:
Best distance  [0.71048258 0.48983359 0.23958833 0.06644133]
Worst distance  [0.70703547 0.49073324 0.2548498  0.05824025]
--------------------------------------------------------------------
Worst similarity  [0.4987841  0.50045874 0.51543314 0.46711192]
Rank to worst similarity  [4, 1, 2, 3]
--------------------------------------------------------------------
Best similarity  [0.5012159  0.49954126 0.48456686 0.53288808]
Rank to worst similarity  [3, 2, 1, 4]
